In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from copy import deepcopy

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

from classes.agent import Agent
from classes.world import World

from classes.utils import *
from classes.news import News
from classes.agent import AgentState

In [24]:
def cascadesize(world):
    return sum([1 if a.is_active() else 0 for a in world.agents.values()])

def measure_cascadesize(world, starting, max_iter):
    r = np.zeros(max_iter)
    for a in starting.keys():
        a.states[starting[a]] = AgentState.ACTIVE
    r[0] = cascadesize(world)
    for iter in range(1, max_iter):
        x = world.update(verbose=True)
        if x == 0:
            for k in range(iter, max_iter):
                r[k] = r[k-1]
                break
        r[iter] = cascadesize(world)
    return r

In [25]:
def comp_starting_sets(worlds, sets, max_iter = 50):
    ccs = []
    for s, w in zip(sets, worlds):
        ccs.append(measure_cascadesize(w, s, max_iter))
    return ccs

In [26]:
sensation_news = 0.5
decay_parameter_news = 0
n1 = News(0, sensation_news, decay_parameter_news)
news1 = {n1.name: n1}
n2 = News(0, sensation_news, decay_parameter_news)
news2 = {n2.name: n2}

number_agents = 100
threshold = 0.5
independence = 0.15
g1 = create_graph(number_agents)
g2 = deepcopy(g1)

agents1 = construct_agent_constant_parameters(number_agents, threshold, independence, news1, g1)
agents2 = construct_agent_constant_parameters(number_agents, threshold, independence, news2, g2)
w1 = World(agents1, news1, g1)
w2 = World(agents2, news2, g2)

In [ ]:
k = 10
kset = approx_most_influential(w1, k, sample_size=25, verbose=False) #THIS TAKES SOME TIME !!!
s1 = {}
for agent in kset:
    s1[agent] = 0
degree_centrality = []
for _ in range(k):
    best = None
    bdeg = -1
    for a in w2.graph.nodes():
        if w2.agents[a] in degree_centrality:
            continue
        d = w2.graph.out_degree(a)
        if d > bdeg:
            best = w2.agents[a]
    degree_centrality.append(best)
s2 = {}
for agent in degree_centrality:
    s2[agent] = 0
print("greedy approx    ", [a.name for a in kset])
print("degree centrality", [a.name for a in degree_centrality])

In [ ]:
n = 50
ccs = comp_starting_sets([w1, w2], [s1, s2], 50)
for cs in ccs:
    print(cs)

In [ ]:
t = np.arange(0, n, 1)
cs1, cs2 = ccs
plt.plot(t, cs1, 'r', t, cs2, 'b')
plt.grid()
plt.show()